In [2]:
# Jupyter notebook related
%reload_ext autoreload
%autoreload 2

In [8]:
import os
import geopandas as gpd
from dask import delayed

# from elogs import Elogs, ElogsTask

with open('../../../connstr_vegteam') as f:
    connect_str = f.read()
container_name = 'evoland'

locs_fn = "../../../locations_v2.csv"

In [9]:
from satio_pc.utils.azure import AzureBlobReader

azure = AzureBlobReader(connect_str,
                        container_name)
if not azure.container_client.exists():
    azure.container_client.create_container()

In [15]:
files = azure.list_files()

In [16]:
len(files)

1211

In [3]:
# from satio_pc.sentinel2 import query_l2a_items, filter_corrupted_items

# from stackstac.stack import items_to_plain, prepare_items, to_coords, to_attrs
# from stackstac.to_dask import asset_table_to_reader_and_window, fetch_raster_window

# import rasterio.windows
# from rasterio.enums import Resampling

# from satio_pc.utils import parallelize

# from stackstac.rio_reader import AutoParallelRioReader, LayeredEnv


# def query_l2a_items(tile,
#                     start_date,
#                     end_date,
#                     max_cloud_cover,
#                     filter_corrupted):
#     import pystac_client
#     import planetary_computer
#     from satio_pc.extension import ESAWorldCoverTimeSeries  # noqa register extension

#     catalog = pystac_client.Client.open(
#         "https://planetarycomputer.microsoft.com/api/stac/v1",
#         modifier=planetary_computer.sign_inplace,
#     )

#     time_range = f"{start_date}/{end_date}"

#     query_params = {"eo:cloud_cover": {"lt": max_cloud_cover},
#                     "s2:mgrs_tile": {"eq": tile}}

#     search = catalog.search(collections=["sentinel-2-l2a"],
#                             datetime=time_range,
#                             query=query_params)
#     items = search.item_collection()

#     if filter_corrupted:
#         items = filter_corrupted_items(items)

#     return items


# def load_reader_table_items(reader_table, spec, fill_value, dtype):
#     current_window = rasterio.windows.from_bounds(*spec.bounds, transform=spec.transform)

#     output = np.broadcast_to(
#         np.array(fill_value, dtype),
#         reader_table.shape + (int(current_window.height), int(current_window.width)),
#     )

#     all_empty: bool = True
#     for index, entry in np.ndenumerate(reader_table):
#         if entry:
#             reader, asset_window = entry
#             # Only read if the window we're fetching actually overlaps with the asset
#             data = reader.read(current_window)

#             if all_empty:
#                 # Turn `output` from a broadcast-trick array to a real array, so it's writeable
#                 if (
#                     np.isnan(data)
#                     if np.isnan(fill_value)
#                     else np.equal(data, fill_value)
#                 ).all():
#                     # Unless the data we just read is all empty anyway
#                     continue
#                 output = np.array(output)
#                 all_empty = False

#             output[index] = data
    
#     return output


# def load_reader_table_items_thread_pool(reader_table, spec, fill_value, dtype, max_workers=20):
#     current_window = rasterio.windows.from_bounds(*spec.bounds, transform=spec.transform)

#     output = np.broadcast_to(
#         np.array(fill_value, dtype),
#         reader_table.shape + (int(current_window.height), int(current_window.width)),
#     )
#     output = np.array(output)
    
#     def _read(index_entry):
#     # for index, entry in np.ndenumerate(reader_table):
#         index, entry = index_entry
#         # if entry:
#         #     print(entry)
#         #     return
#         reader, asset_window = entry
#         # Only read if the window we're fetching actually overlaps with the asset
#         data = reader.read(current_window)        
#         print(data.sum())
#         output[index] = data

#     _ = parallelize(
#         _read,
#         list(np.ndenumerate(reader_table)),
#         max_workers=max_workers,
#         progressbar=False)

#     return output



# def load_items(
#     items,
#     assets,
#     bounds,
#     epsg,
#     resolution,
#     dtype,
#     xy_coords='center',
#     fill_value=0,
#     resampling=Resampling.nearest,
#     max_workers=20,
#     ):
    
#     plain_items = items_to_plain(items)
#     plain_items = sorted(
#         plain_items,
#         key=lambda item: item["properties"].get("datetime", "") or "",
#         reverse=False,
#     )

#     asset_table, spec, asset_ids, plain_items = prepare_items(
#         plain_items,
#         assets=assets,
#         epsg=epsg,
#         resolution=resolution,
#         bounds=bounds,
#         bounds_latlon=None,
#         snap_bounds=True,
#     )

#     reader_table = asset_table_to_reader_and_window(
#         asset_table,
#         spec,
#         resampling,
#         dtype=dtype,
#         fill_value=fill_value,
#         rescale=False,
#         gdal_env=None,
#         errors_as_nodata=(),
#         reader=AutoParallelRioReader)
    
#     if max_workers == -1:
#         arr = load_reader_table_items(reader_table, spec, fill_value, dtype)
#     else:
#         arr = load_reader_table_items_thread_pool(reader_table, spec, fill_value, dtype, max_workers)
        
#     return xr.DataArray(
#         arr,
#         *to_coords(
#             plain_items,
#             asset_ids,
#             spec,
#             xy_coords=xy_coords,
#             properties=True,
#             band_coords=True,
#         ),
#         attrs=to_attrs(spec),
#         name="stac-data",
#     )


# class S2TileReader:
    
#     def __init__(self,
#                  tile,
#                  start_date,
#                  end_date,
#                  max_cloud_cover=90,
#                  filter_corrupted=True):
    
#         self._tile = tile
#         self._start_date = start_date
#         self._end_date = end_date
        
#         self._max_cloud_cover = max_cloud_cover
#         self._filter_corrupted = True
        
#         self._items = None
        
#     @property
#     def items(self):
#         if self._items is None:
#             self._items = query_l2a_items(self._tile,
#                                           self._start_date,
#                                           self._end_date,
#                                           self._max_cloud_cover,
#                                           self._filter_corrupted)
#         return self._items

#     def assets(self, bands):
        
#         assets_10m = set(['B02', 'B03', 'B04', 'B08'])
#         assets_20m = set(['B05', 'B06', 'B07', 'B8A', 'B11', 'B12'])
#         assets_60m = set(['B01', 'B09'])
        
#         sbands = set(bands)
        
#         dtype = np.uint16
        
#         if len(sbands & assets_10m):
#             resolution = 10
#         elif len(sbands & assets_20m):
#             resolution = 20
#         elif len(sbands & assets_60m):
#             resolution = 60
#         elif bands == ['SCL']:
#             dtype = np.uint8
#         else:
#             raise ValueError(f"Bands is a mix of resolutions or not recognized: {bands}")
                             
#         return bands, dtype, resolution
    
#     def read(self,
#              bounds,
#              epsg,
#              bands,
#              max_workers=20):
        
#         assets, dtype, resolution = self.assets(bands)      
                             
#         darr = load_items(
#             self.items,
#             assets,
#             bounds,
#             f'EPSG:{epsg}',
#             resolution,
#             dtype,
#             xy_coords='center',
#             fill_value=0,
#             resampling=Resampling.nearest,
#             max_workers=max_workers,
#             )
#         return darr

In [3]:
import numpy as np
import pandas as pd

from satio_pc.grid import get_blocks_gdf

In [10]:
locs = pd.read_csv(locs_fn)

In [5]:
loc = locs.sample(1).iloc[0]

In [6]:
blocks = get_blocks_gdf(['31UFS'])

In [54]:
year = 2020

start_date = f'{year}-01-01'
end_date = f'{year}-09-01'

max_cloud_cover = 90


def read_block(block):
    from satio_pc.reader import S2TileReader
    tile = block.tile
    epsg = block.epsg
    # bounds = loc.xmin, loc.ymin, loc.xmax, loc.ymax
    bounds = block.bounds

    bands = ['B08']

    reader = S2TileReader(tile,
                          start_date,
                          end_date,
                          max_cloud_cover)
    darr = reader.read(bounds, epsg, bands, max_workers=20)
    return darr


def read_loc(loc, band):
    from satio_pc.reader import S2TileReader
    tile = loc.tile
    epsg = loc.epsg
    bounds = loc.xmin, loc.ymin, loc.xmax, loc.ymax

    bands = [band]

    reader = S2TileReader(tile,
                          start_date,
                          end_date,
                          max_cloud_cover)
    darr = reader.read(bounds, epsg, bands, max_workers=20)
    
    
    return darr


In [56]:
band = 'B08'
darr = read_loc(loc, band)

2023-07-14 13:36:46.616 | WARNING  | satio_pc.sentinel2:filter_corrupted_items:57 - Discarding 1 / 65 corrupted products: ['S2A_MSIL2A_20200424T034541_R104_T47NMG_20200921T210310']
/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [ ]:
darr.ewc.composite(

In [14]:
# loc = blocks.iloc[55]

In [15]:
# after starting the cluster (below)
args = list(blocks.itertuples())[:5]

In [16]:
args = list(locs.iloc[:10].itertuples())

In [17]:
import dask

extract_delayed = dask.delayed(read_loc)

lazy_results = []

for ag in args:
    lazy_result = extract_delayed(ag)
    lazy_results.append(lazy_result)

# futures = dask.persist(*lazy_results)  # trigger computation in the background

results = dask.compute(*lazy_results)




/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(
/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(
/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdep

In [6]:
from satio_pc.geotiff import slash_tile



In [7]:
len(f)

251

In [21]:
azure.delete_file('evotrain/v2/year/59/F/LB/evotrain_v2_2020_59FLB_018_62.tif')

In [3]:
settings = {
        "max_cloud_cover": 90,
        "composite": {"freq": 10,
                      "window": 20,
                      "mode": "median"},
        "mask": {"erode_r": 3,
                 "dilate_r": 13,
                 "max_invalid_ratio": 1},
        "bands": ['B02', 'B03', 'B04', 'B08', 'B11', 'B12'],
        "indices": ["ndvi"],
        "percentiles": [10, 25, 50, 75, 90],
    }


In [4]:
import pandas as pd


locs = pd.read_csv(locs_fn)

In [5]:
def extract_loc(loc, year):
    from satio_pc.geotiff import slash_tile
    from satio_pc.extraction import S2Extractor
    from satio_pc.utils.azure import AzureBlobReader
    
    s2 = S2Extractor(settings=settings)

    bounds = loc.xmin, loc.ymin, loc.xmax, loc.ymax
    feats = s2.extract(year, loc.tile, bounds, loc.epsg)

    fn = f"evotrain_v2_{year}_{loc.patch_id}.tif"
    feats.ewc.save_features(fn, bounds, loc.epsg)
    
    azure = AzureBlobReader(connect_str,
                            container_name)
    
    dst_fn = f"evotrain/v2/{year}/{slash_tile(loc.tile)}/{fn}"
    azure.upload_file(fn,
                      dst_fn,
                      overwrite=True)
    
    os.remove(fn)

def extract_loc(loc, year):
    from satio_pc.geotiff import slash_tile
    from satio_pc.extraction import S2Extractor
    
    s2 = S2Extractor(settings=settings)

    bounds = loc.xmin, loc.ymin, loc.xmax, loc.ymax
    # feats = s2.extract(year, loc.tile, bounds, loc.epsg)

    return bounds

In [44]:
# extract_loc(loc, 2020)

In [11]:
import hvplot.xarray  # noqa
import hvplot.pandas  # noqa
import panel as pn  # noqa
import panel.widgets as pnw


feats.interactive.sel(band=pnw.DiscreteSlider).hvplot(
    colormap='plasma',
    aspect=1,
    x='x',
    y='y')

# Cluster setup

In [52]:
# stop clusters
from dask_gateway import Gateway
gateway = Gateway()
clusters_reports = gateway.list_clusters()

clusters = [gateway.stop_cluster(c.name) for c in clusters_reports]

In [51]:
from dask.distributed import PipInstall, Client
import dask_gateway

cluster = dask_gateway.GatewayCluster()
client = cluster.get_client()
cluster.scale(3)
print(client.dashboard_link)

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.8285370bebfa4fb78689341429774d53/status


In [12]:
satio_pc_url = "git+https://github.com/dzanaga/satio-pc.git@main"
# elogs_url = "http://s3-eu-central-1.amazonaws.com/vito-worldcover-public/wheels/elogs-0.1.5-py3-none-any.whl"
plugin = PipInstall(packages=[satio_pc_url],
                    pip_options=["--upgrade"])
client.register_worker_plugin(plugin)

{'tls://10.244.147.79:42219': {'status': 'OK'},
 'tls://10.244.147.80:36813': {'status': 'OK'},
 'tls://10.244.148.181:33059': {'status': 'OK'}}

In [ ]:
# check logs
logs = client.get_worker_logs()
for worker, worker_logs in logs.items():
    print(f"Logs for worker {worker}:")
    for log in worker_logs:
        print(log)
    print()
    print('*'*100)

In [40]:
cluster.shutdown()

In [15]:
args = [(loc, year)
        for year in range(2018, 2023)
        for loc in locs.iloc[:5].itertuples()]

In [16]:
len(args)

25

In [77]:
def extract_loc(loc, year):
    from satio_pc.geotiff import slash_tile
    from satio_pc.extraction import S2Extractor
    from satio_pc.utils.azure import AzureBlobReader
    
    s2 = S2Extractor(settings=settings)

    bounds = loc.xmin, loc.ymin, loc.xmax, loc.ymax
    feats = s2.extract(year, loc.tile, bounds, loc.epsg)

    fn = f"evotrain_v2_{year}_{loc.patch_id}.tif"
    feats.ewc.save_features(fn, bounds, loc.epsg)
    
    azure = AzureBlobReader(connect_str,
                            container_name)
    
    dst_fn = f"evotrain/v2/{year}/{slash_tile(loc.tile)}/{fn}"
    azure.upload_file(fn,
                      dst_fn,
                      overwrite=True)
    
    os.remove(fn)

    
def extract_loc(loc, year):
    from satio_pc.geotiff import slash_tile
    from satio_pc.extraction import S2Extractor
    from satio_pc.sentinel2 import load_l2a
    
    s2 = S2Extractor(settings=settings)
    
    bounds = loc.xmin, loc.ymin, loc.xmax, loc.ymax
    epsg = loc.epsg
    tile = loc.tile
    
    # feats = s2.extract(year, loc.tile, bounds, loc.epsg)
    
    l2a = load_l2a(bounds,
                   epsg,
                   tile,
                   '2020-06-01',
                   '2020-09-01',
                   bands=['B08'],
                   max_cloud_cover=90,
                   filter_corrupted=True)
    
    l2a = l2a[10].compute()
    
    return l2a


def extract_loc_wrap(loc, year):
    import multiprocessing
    args = (loc, year)
    p = multiprocessing.Process(target=extract_loc,
                                args=args)
    p.start()
    p.join()

In [75]:
import stackstac

In [ ]:
stackstac.stack(

In [73]:
import dask

extract_delayed = dask.delayed(extract_loc_wrap)

lazy_results = []

for ag in args[:10]:
    lazy_result = extract_delayed(*ag)
    lazy_results.append(lazy_result)

# futures = dask.persist(*lazy_results)  # trigger computation in the background

results = dask.compute(*lazy_results)

r = results[0][10]

In [74]:
r

<xarray.DataArray 'stackstac-04cee801468489935c1c372c6be17741' (time: 22,
                                                                band: 1,
                                                                y: 128, x: 128)>
dask.array<fetch_raster_window, shape=(22, 1, 128, 128), dtype=uint16, chunksize=(1, 1, 128, 128), chunktype=numpy.ndarray>
Coordinates:
  * time                    (time) datetime64[ns] 2020-06-09T22:20:49.024000 ...
    id                      (time) <U54 'S2B_MSIL2A_20200609T222049_R129_T59F...
  * band                    (band) <U3 'B08'
  * x                       (x) float64 3.794e+05 3.794e+05 ... 3.806e+05
  * y                       (y) float64 4.181e+06 4.181e+06 ... 4.18e+06
    s2:processing_baseline  <U5 '02.12'
Attributes:
    crs:         epsg:EPSG:32759
    transform:   | 10.00, 0.00, 379360.00|\n| 0.00,-10.00, 4180840.00|\n| 0.0...
    resolution:  10.0
    bounds:      (379360.0, 4179560.0, 380640.0, 4180840.0)

In [61]:
arr = results[0][10].compute()

In [62]:
arr

<xarray.DataArray 'stackstac-afdbce8ec64c12559d1cb2f531c9509f' (time: 22,
                                                                band: 1,
                                                                y: 128, x: 128)>
array([[[[10224, 10304, 10416, ...,  7612,  7524,  7648],
         [10248, 10400, 10448, ...,  7496,  7592,  7636],
         [10352, 10384, 10256, ...,  7364,  7484,  7592],
         ...,
         [ 6764,  6872,  6752, ...,  6532,  6520,  6468],
         [ 6632,  6612,  6756, ...,  6612,  6540,  6632],
         [ 6740,  6684,  6808, ...,  6676,  6496,  6612]]],


       [[[  969,   978,   900, ...,     1,    15,    61],
         [ 1042,  1017,   972, ...,     1,    41,    28],
         [ 1076,  1102,   976, ...,    12,    68,    67],
         ...,
         [ 1176,  1300,  1260, ...,  1724,  1888,  2372],
         [ 1066,  1096,  1248, ...,  1856,  2104,  2670],
         [ 1074,  1100,  1270, ...,  2010,  2316,  2806]]],


       [[[ 3312,  3338,  3164, ...,  9504, 10312, 10664],
         [ 3548,  3572,  3360, ..., 10232, 10280, 10192],
...
         [13768, 14176, 14424, ...,  2158,  2250,  2230],
         [13656, 13888, 14144, ...,  1996,  2078,  2132]]],


       [[[ 1974,  2016,  2040, ...,  4094,  4094,  4140],
         [ 1942,  2020,  2108, ...,  4072,  4094,  4132],
         [ 1936,  2030,  2124, ...,  4072,  4112,  4140],
         ...,
         [ 2526,  2484,  2526, ...,  2120,  2136,  2110],
         [ 2492,  2512,  2486, ...,  2040,  2054,  2086],
         [ 2436,  2466,  2486, ...,  2018,  2040,  2092]]],


       [[[ 8656,  8608,  8800, ...,  5464,  5500,  5440],
         [ 8416,  8392,  8624, ...,  5340,  5332,  5376],
         [ 8328,  8280,  8464, ...,  5464,  5464,  5464],
         ...,
         [11240, 11520, 11608, ..., 10096,  9928,  9808],
         [11096, 11200, 11256, ...,  9880,  9712,  9688],
         [10896, 10936, 10896, ...,  9680,  9560,  9592]]]], dtype=uint16)
Coordinates:
  * time                    (time) datetime64[ns] 2020-06-09T22:20:49.024000 ...
    id                      (time) <U54 'S2B_MSIL2A_20200609T222049_R129_T59F...
  * band                    (band) <U3 'B08'
  * x                       (x) float64 3.794e+05 3.794e+05 ... 3.806e+05
  * y                       (y) float64 4.181e+06 4.181e+06 ... 4.18e+06
    s2:processing_baseline  <U5 '02.12'
Attributes:
    crs:         epsg:EPSG:32759
    transform:   | 10.00, 0.00, 379360.00|\n| 0.00,-10.00, 4180840.00|\n| 0.0...
    resolution:  10.0
    bounds:      (379360.0, 4179560.0, 380640.0, 4180840.0)

In [ ]:
feats = dask.compute(lazy_results[0])

In [40]:
worker = 'tls://10.244.150.23:40551'

In [41]:
f = client.submit(extract_loc, args[:3], workers=worker)

In [42]:
f

<Future: error, key: extract_loc-2a96274995de5f5cb82147bc23c2889c>

In [34]:
workers = client.scheduler_info()['workers']

In [30]:
dask.compute(f[0])

(<Future: error, key: extract_loc-f91a193c9d4fe0a3b4707ac879b0aa85>,)

In [ ]:
feats

In [ ]:
f = lazy_results[1].compute()

In [ ]:
futures = client.map(extract_s2, args[:6])

In [ ]:
f = futures[5]
f

In [ ]:
f.result()

In [ ]:
@dask.delayed
def test(n):
    try:
        import satio_pc
        s2grid = satio_pc.layers.load('s2grid')
        return s2grid.shape[0]
    except:
        return 0

In [ ]:
nn = list(range(30))

In [ ]:
futures = client.map(test, nn)

In [ ]:
f = futures[10]

In [ ]:
f.result().compute()

In [ ]:
client.

In [ ]:
client

In [ ]:
# list clusters
from dask_gateway import Gateway
gateway = Gateway()
gateway.list_clusters()

In [ ]:
# # plugin = PipInstall(packages=["git+https://github.com/dzanaga/satio-pc/tree/main/satio_pc"], pip_options=["--upgrade"])
# # client.register_worker_plugin(plugin)
# satio_pc_url = "git+https://github.com/dzanaga/satio-pc.git@main"
# elogs_url = "http://s3-eu-central-1.amazonaws.com/vito-worldcover-public/wheels/elogs-0.1.5-py3-none-any.whl"
# plugin = PipInstall(packages=[satio_pc_url,
#                               elogs_url],
#                     pip_options=["--upgrade"])
# client.register_worker_plugin(plugin)

# print(client.dashboard_link)

In [ ]:
# plugin = PipInstall(packages=["git+https://github.com/dzanaga/satio-pc/tree/main/satio_pc"], pip_options=["--upgrade"])
# client.register_worker_plugin(plugin)
satio_pc_url = "git+https://github.com/dzanaga/satio-pc.git@main"
elogs_url = "http://s3-eu-central-1.amazonaws.com/vito-worldcover-public/wheels/elogs-0.1.5-py3-none-any.whl"
plugin = PipInstall(packages=[satio_pc_url],
                    pip_options=["--upgrade"])
client.register_worker_plugin(plugin)

print(client.dashboard_link)

In [ ]:
import dask
lazy_results = []

for ag in args[:20]:
    lazy_result = dask.delayed(extract_s2)(ag)
    lazy_results.append(lazy_result)

futures = dask.persist(*lazy_results)  # trigger computation in the background

In [ ]:
futures[0].compute()

In [ ]:
workers = client.scheduler_info()['workers']
tasks = [client.submit(extract_s2, arg, workers=worker)
         for arg, worker in zip(args[:16], workers)]

In [ ]:
tasks[0]

In [ ]:
# test first block

block = next(blocks_gdf.iloc[[10]].itertuples())
year = 2021

In [ ]:
extractor = S2BlockExtractor(block.tile,
                             block.block_id,
                             year,
                             settings,
                             bands=bands,
                             indices=indices,
                             percentiles=percentiles,
                             output_folder='.',
                             connection_str=connect_str,
                             container_name=container_name)

In [ ]:
data, fn, bounds, epsg = extractor._extract_s2()

In [ ]:
fn

In [ ]:
data.ewc.save_features(fn, bounds, epsg)

# debug extract function

In [ ]:
import xarray as xr
from loguru import logger

from satio_pc.sentinel2 import load_l2a, preprocess_l2a
from satio_pc.preprocessing.clouds import preprocess_scl
from satio_pc.grid import get_blocks_gdf

# year = self.year
# tile = self.tile
# block_id = self.block_id
self = extractor

start_date = f'{year}-01-01'
end_date = f'{year + 1}-01-01'
max_cloud_cover = self._settings['l2a']['max_cloud_cover']

# blocks = get_blocks_gdf([tile])
# block = blocks[blocks.block_id == block_id].iloc[0]

s2_dict = load_l2a(block.bounds,
                   block.epsg,
                   block.tile,
                   start_date,
                   end_date,
                   bands=self._bands,
                   max_cloud_cover=max_cloud_cover)

# mask preparation
mask_settings = self._settings['l2a']['mask']
scl = preprocess_scl(s2_dict['scl'],
                     **mask_settings)

scl20_mask = scl.mask
scl20_aux = scl.aux

s2 = preprocess_l2a(s2_dict,
                    scl20_mask,
                    start_date,
                    end_date,
                    composite_freq=self._settings[
                        'l2a']['composite']['freq'],
                    composite_window=self._settings[
                        'l2a']['composite']['window'],
                    composite_mode=self._settings[
                        'l2a']['composite']['mode'])

s2_indices = self._indices

# compute indices
s2_vi = s2.ewc.indices(s2_indices)

In [ ]:
s2_vi.ewc.show(band='ndvi')

In [ ]:
# percentiles sensors and vis
q = self._percentiles
ps = [s.ewc.percentile(q, name_prefix='s2') for s in (s2, s2_vi)]

# fix time to same timestamp (only 1) to avoid concat issues
# (different compositing settings for s2 and s1)
for p in ps:
    p['time'] = ps[0].time

# scl aux 10m
scl10_aux = scl20_aux.ewc.rescale(scale=2, order=1)
scl10_aux['time'] = ps[0].time

final = xr.concat(ps + [scl10_aux], dim='band')
final.name = 'satio-features-s2'

logger.info("Computing features stack")
final = final.persist()
final = final.squeeze()

output_folder = Path(self.block_folder)
fn = output_folder / \
    f'{final.name}_{tile}_{block.block_id:03d}_{year}.tif'

In [ ]:
s2 = preprocess_l2a(s2_dict,
                    scl20_mask,
                    start_date,
                    end_date,
                    composite_freq=settings['l2a']['composite']['freq'],
                    composite_window=settings['l2a']['composite'][
                        'window'],
                    tmpdir=tmpdir.name)

s2_indices = indices

# compute indices
s2_vi = s2.ewc.indices(s2_indices)

In [ ]:
s2_vi.ewc.plot(band='ndvi', vmin=-1, vmax=1)

In [ ]:
import hvplot.xarray  # noqa
import hvplot.pandas  # noqa
import panel as pn  # noqa
import panel.widgets as pnw

im = s2
band = im.band[0]
im = im.sel(band=band)

im.interactive.sel(time=pnw.DiscreteSlider).hvplot(
    clim=(0, 0.2), colormap='plasma', aspect=1)

bug in the interpolate function? there are 0s in the s2 ts and nans in the ndvi ts
ndvi is saved as mostly nan?

### debug preprocess l2a

In [ ]:
import xarray as xr
import dask.array as da
from pyproj.crs import CRS
from loguru import logger
import tempfile
from satio_pc.sentinel2 import load_l2a, preprocess_l2a
from satio_pc.preprocessing.clouds import preprocess_scl
from satio_pc.grid import get_blocks_gdf, tile_to_epsg


year = 2021
tile = block.tile
block_id = block.block_id

start_date = f'{year}-01-01'
end_date = f'{year + 1}-01-01'
max_cloud_cover = settings['l2a']['max_cloud_cover']

s2_dict = load_l2a(block.bounds,
                   block.epsg,
                   block.tile,
                   start_date,
                   end_date,
                   bands=bands,
                   max_cloud_cover=max_cloud_cover)

# preprocess s2
tmpdir = tempfile.TemporaryDirectory(prefix='ewc_tmp-',
                                     dir='.')

# mask preparation
mask_settings = settings['l2a']['mask']
scl = preprocess_scl(s2_dict['scl'],
                     **mask_settings)

scl20_mask = scl.mask
scl20_aux = scl.aux

In [ ]:
# s2 = preprocess_l2a(s2_dict,
#                     scl20_mask,
#                     start_date,
#                     end_date,
#                     composite_freq=settings['l2a']['composite']['freq'],
#                     composite_window=settings['l2a']['composite'][
#                         'window'],
#                     tmpdir=tmpdir.name)

ds_dict = s2_dict
clouds_mask = scl20_mask
composite_freq=settings['l2a']['composite']['freq']
composite_window=settings['l2a']['composite']['window']

ds10_block = ds_dict[10]
ds20_block = ds_dict[20]
scl20_block = clouds_mask

In [ ]:
ds10_block.ewc.rgb(vmax=2000)

In [ ]:
from satio_pc.preprocessing.timer import FeaturesTimer

In [ ]:
ds10_block = ds_dict[10].sel(band=['B04', 'B08'])
# ds20_block = ds_dict[20]
scl20_block = clouds_mask

timer10 = FeaturesTimer(10, 'l2a')
timer20 = FeaturesTimer(20, 'l2a')

# download
logger.info("Loading block data")
timer10.load.start()
ds10_block = ds10_block.ewc.persist_chunk()
timer10.load.stop()

timer20.load.start()
# ds20_block = ds20_block.ewc.persist_chunk()
scl20_block = scl20_block.ewc.persist_chunk()
scl10_block = scl20_block.ewc.rescale(scale=2,
                                      order=0)
scl10_block = scl10_block.ewc.persist_chunk()
timer20.load.stop()

# 10m
# mask clouds
timer10.composite.start()
ds10_block_masked = ds10_block.ewc.mask(
    scl10_block).ewc.persist_chunk()

logger.info("Compositing 10m block data")
# composite
ds10_block_comp = ds10_block_masked.ewc.composite(
    freq=composite_freq,
    window=composite_window,
    start=start_date,
    end=end_date).ewc.persist_chunk()
timer10.composite.stop()

logger.info("Interpolating 10m block data")
# interpolation
timer10.interpolate.start()
ds10_block_interp = ds10_block_comp.ewc.interpolate(
).ewc.persist_chunk()
timer10.interpolate.stop()

# 20m
# mask
# timer20.composite.start()
# ds20_block_masked = ds20_block.ewc.mask(
#     scl20_block).ewc.persist_chunk()

# logger.info("Compositing 20m block data")
# # composite
# ds20_block_comp = ds20_block_masked.ewc.composite(
#     freq=composite_freq,
#     window=composite_window,
#     start=start_date,
#     end=end_date).ewc.persist_chunk()
# timer20.composite.stop()

# logger.info("Interpolating 20m block data")
# # interpolation
# timer20.interpolate.start()
# ds20_block_interp = ds20_block_comp.ewc.interpolate(
# ).ewc.persist_chunk()
# timer20.interpolate.stop()

# logger.info("Merging 10m and 20m series")
# # merging to 10m cleaned data
# ds20_block_interp_10m = ds20_block_interp.ewc.rescale(scale=2,
#                                                       order=1,
#                                                       nodata_value=0)
dsm10 = xr.concat([ds10_block_interp],
                  dim='band')

In [ ]:
show(ds10_block_interp)

In [ ]:
if reflectance:
    dsm10 = dsm10.astype(np.float32) / 10000

dsm10.attrs = ds10_block.attrs

for t in timer10, timer20:
    t.load.log()
    t.composite.log()
    t.interpolate.log()

for t in timer10, timer20:
    t.log()

dsm10 = dsm10.ewc.persist_chunk()

In [ ]:
def show(ds, band=None, vmin=None, vmax=None,
         colormap='plasma', **kwargs):
    import hvplot.xarray  # noqa
    import hvplot.pandas  # noqa
    import panel as pn  # noqa
    import panel.widgets as pnw

    im = ds
    band = im.band[0] if band is None else band
    im = im.sel(band=band)
    return im.interactive.sel(time=pnw.DiscreteSlider).hvplot(
        clim=(vmin, vmax),
        colormap=colormap,
        aspect=1,
        x='x',
        y='y',
        **kwargs)

In [ ]:
# show(ds10_block)

In [ ]:
def preprocess_l2a(ds_dict,
                   clouds_mask,
                   start_date,
                   end_date,
                   composite_freq=10,
                   composite_window=20,
                   composite_mode='median',
                   reflectance=True,
                   tmpdir='.'):

    ds10_block = ds_dict[10]
    ds20_block = ds_dict[20]
    scl20_block = clouds_mask

    timer10 = FeaturesTimer(10, 'l2a')
    timer20 = FeaturesTimer(20, 'l2a')

    # download
    logger.info("Loading block data")
    timer10.load.start()
    ds10_block = ds10_block.persist()
    timer10.load.stop()

    timer20.load.start()
    ds20_block = ds20_block.persist()
    scl20_block = scl20_block.persist()
    scl10_block = scl20_block.ewc.rescale(scale=2,
                                          order=0)
    scl10_block = scl10_block.persist()
    timer20.load.stop()

    # 10m
    # mask clouds
    timer10.composite.start()
    ds10_block_masked = ds10_block.ewc.mask(
        scl10_block).persist()

    logger.info("Compositing 10m block data")
    # composite
    ds10_block_comp = ds10_block_masked.ewc.composite(
        freq=composite_freq,
        window=composite_window,
        start=start_date,
        end=end_date).persist()
    timer10.composite.stop()

    logger.info("Interpolating 10m block data")
    # interpolation
    timer10.interpolate.start()
    ds10_block_interp = ds10_block_comp.ewc.interpolate(
    ).persist()
    timer10.interpolate.stop()

    # 20m
    # mask
    timer20.composite.start()
    ds20_block_masked = ds20_block.ewc.mask(
        scl20_block).persist()

    logger.info("Compositing 20m block data")
    # composite
    ds20_block_comp = ds20_block_masked.ewc.composite(
        freq=composite_freq,
        window=composite_window,
        start=start_date,
        end=end_date).persist()
    timer20.composite.stop()

    logger.info("Interpolating 20m block data")
    # interpolation
    timer20.interpolate.start()
    ds20_block_interp = ds20_block_comp.ewc.interpolate(
    ).persist()
    timer20.interpolate.stop()

    logger.info("Merging 10m and 20m series")
    # merging to 10m cleaned data
    ds20_block_interp_10m = ds20_block_interp.ewc.rescale(scale=2,
                                                          order=1,
                                                          nodata_value=0)
    dsm10 = xr.concat([ds10_block_interp,
                       ds20_block_interp_10m],
                      dim='band')

    if reflectance:
        dsm10 = dsm10.astype(np.float32) / 10000

    dsm10.attrs = ds10_block.attrs

    for t in timer10, timer20:
        t.load.log()
        t.composite.log()
        t.interpolate.log()

    for t in timer10, timer20:
        t.log()

    dsm10 = dsm10.ewc.persist_chunk()

    return dsm10